In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
from IPython.display import clear_output
import time

# =============================================================================
# CONSTANTS AND CONFIGURATION
# =============================================================================

MARKET_RATES = {
    'DEFAULT': {
        'growth_rate': 0.035, 'risk_free_rate': 0.032, 'market_risk_premium': 0.05,
        'terminal_growth': 0.022, 'discount_rate': 0.082, 'maintenance_capex_ratio': 0.28,
        'min_owner_earnings_yield': 0.045, 'strong_owner_earnings_yield': 0.08,
        'min_growth_rate': 0.01, 'max_earnings_growth': 0.5, 'max_fcf_growth': 0.3,
        'dcf_years': 10, 'default_beta': 1.0, 'min_discount_premium': 0.01,
        'graham_constant': 22.5, 'graham_base_pe': 8.5
    },
    'SG': {
        'growth_rate': 0.03, 'risk_free_rate': 0.025, 'market_risk_premium': 0.045,
        'terminal_growth': 0.02, 'discount_rate': 0.07, 'maintenance_capex_ratio': 0.25,
        'min_owner_earnings_yield': 0.04, 'strong_owner_earnings_yield': 0.07,
        'min_growth_rate': 0.01, 'max_earnings_growth': 0.4, 'max_fcf_growth': 0.25,
        'dcf_years': 10, 'default_beta': 0.9, 'min_discount_premium': 0.01,
        'graham_constant': 22.5, 'graham_base_pe': 8.5
    },
    'HK': {
        'growth_rate': 0.035, 'risk_free_rate': 0.032, 'market_risk_premium': 0.05,
        'terminal_growth': 0.022, 'discount_rate': 0.082, 'maintenance_capex_ratio': 0.28,
        'min_owner_earnings_yield': 0.045, 'strong_owner_earnings_yield': 0.08,
        'min_growth_rate': 0.01, 'max_earnings_growth': 0.5, 'max_fcf_growth': 0.3,
        'dcf_years': 10, 'default_beta': 1.0, 'min_discount_premium': 0.01,
        'graham_constant': 22.5, 'graham_base_pe': 8.5
    },
    'CN': {
        'growth_rate': 0.055, 'risk_free_rate': 0.028, 'market_risk_premium': 0.065,
        'terminal_growth': 0.03, 'discount_rate': 0.095, 'maintenance_capex_ratio': 0.35,
        'min_owner_earnings_yield': 0.06, 'strong_owner_earnings_yield': 0.10,
        'min_growth_rate': 0.02, 'max_earnings_growth': 0.6, 'max_fcf_growth': 0.4,
        'dcf_years': 10, 'default_beta': 1.2, 'min_discount_premium': 0.015,
        'graham_constant': 22.5, 'graham_base_pe': 8.5
    },
    'US': {
        'growth_rate': 0.04, 'risk_free_rate': 0.042, 'market_risk_premium': 0.055,
        'terminal_growth': 0.025, 'discount_rate': 0.085, 'maintenance_capex_ratio': 0.3,
        'min_owner_earnings_yield': 0.04, 'strong_owner_earnings_yield': 0.07,
        'min_growth_rate': 0.01, 'max_earnings_growth': 0.5, 'max_fcf_growth': 0.3,
        'dcf_years': 10, 'default_beta': 1.0, 'min_discount_premium': 0.01,
        'graham_constant': 22.5, 'graham_base_pe': 8.5
    }
}

BENCHMARKS = {
    'roe_strong': 0.15, 'roe_weak': 0.08, 'debt_to_equity_high': 1.0, 'debt_to_equity_low': 0.3,
    'fcf_yield_strong': 0.08, 'fcf_yield_weak': 0.03, 'revenue_growth_strong': 0.15,
    'revenue_growth_slow': 0.03, 'operating_margin_high': 0.20, 'operating_margin_low': 0.05,
    'margin_of_safety_strong': 0.15, 'current_ratio_min': 1.5, 'current_ratio_max': 3.0,
    'quick_ratio_min': 1.0
}

# =============================================================================
# UTILITY FUNCTIONS
# =============================================================================

def safe_divide(numerator, denominator, default=None):
    """Safely divide two values, handling None and zero denominator"""
    if numerator is None or denominator is None or denominator == 0:
        return default
    try:
        return numerator / denominator
    except (TypeError, ZeroDivisionError):
        return default

def get_market_rates(company_stats):
    """Get market-specific rates based on company country or exchange"""
    if not company_stats:
        return MARKET_RATES['DEFAULT']
    
    country = str(company_stats.get('country', '')).upper()
    currency = str(company_stats.get('currency', '')).upper()
    
    search_text = country + currency
    
    if any(indicator in search_text for indicator in ['SINGAPORE', 'SG', 'SGD']):
        return MARKET_RATES['SG']
    elif any(indicator in search_text for indicator in ['HONG KONG', 'HK', 'HKG', 'HKD']):
        return MARKET_RATES['HK']
    elif any(indicator in search_text for indicator in ['CHINA', 'CN', 'RMB', 'CNY']):
        return MARKET_RATES['CN']
    elif any(indicator in search_text for indicator in ['USA', 'US', 'UNITED STATES', 'USD']):
        return MARKET_RATES['US']
    else:
        return MARKET_RATES['DEFAULT']

def format_value(value, format_type='currency', decimals=2):
    """Universal formatting function for different value types"""
    if value is None or value == 'N/A' or pd.isna(value):
        return 'N/A'
    
    try:
        value = float(value)
        
        if format_type == 'currency':
            if abs(value) >= 1e12: return f"${value/1e12:.{decimals}f}T"
            elif abs(value) >= 1e9: return f"${value/1e9:.{decimals}f}B"
            elif abs(value) >= 1e6: return f"${value/1e6:.{decimals}f}M"
            elif abs(value) >= 1e3: return f"${value/1e3:.{decimals}f}K"
            else: return f"${value:.{decimals}f}"
        elif format_type == 'percentage':
            return f"{value*100:.{decimals}f}%"
        elif format_type == 'ratio':
            return f"{value:.{decimals}f}"
        elif format_type == 'growth':
            return f"{value*100:.{decimals}f}%"
        elif format_type == 'dte':
            return f"{value/100:.{decimals}f}"
        elif format_type == 'eps':
            return f"${value:.{decimals}f}"
    except (ValueError, TypeError):
        return 'N/A'

def format_trend_list(trend_list, format_type='ratio', decimals=2):
    """Format a list of trend values into a readable string"""
    if not trend_list or trend_list == 'N/A':
        return 'N/A'
    
    try:
        formatted_values = []
        for value in trend_list:
            if value is None:
                formatted_values.append('N/A')
            else:
                formatted_values.append(format_value(value, format_type, decimals))
        
        return " ← ".join(formatted_values)
    except Exception as e:
        print(f"Error formatting trend list: {e}")
        return 'N/A'

# =============================================================================
# DATA FETCHING AND PROCESSING
# =============================================================================

def get_company_data(ticker):
    """Fetch all company data in one function"""
    try:
        stock = yf.Ticker(ticker)
        company_stats = stock.info
        
        # Fetch financial statements for trend analysis
        income_statement = stock.financials
        balance_sheet = stock.balance_sheet
        cash_flow = stock.cashflow
        
        # Combine financial statements
        financials_statement = pd.concat([income_statement, balance_sheet, cash_flow], axis=0)
        
        # Reorder columns - format the columns to YYYY format and rearrange for the newest year first
        if not financials_statement.empty:
            # Convert column names to proper datetime and sort
            financials_statement.columns = pd.to_datetime(financials_statement.columns).strftime('%Y')
            financials_statement = financials_statement.sort_index(axis=1, ascending=False)
            # Format columns to YYYY only if desired
            financials_statement.columns = financials_statement.columns.str.split('-').str[0]
        
        return company_stats, financials_statement
        
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None, None

def calculate_trend_metrics(financial_data):
    """Calculate trend metrics from financial statements with historical values"""
    trend_metrics = {}
    
    try:
        # Only process first 4 years (most recent 4 columns)
        available_years = financial_data.columns[:4]  # Take first 4 columns only
        
        # Financial health ratios - already in list format
        if ('Current Assets' in financial_data.index and 
            'Current Liabilities' in financial_data.index):
            
            current_ratio_list = []
            
            # Loop through only first 4 years
            for col in available_years:
                current_assets = financial_data.loc['Current Assets', col]
                current_liabilities = financial_data.loc['Current Liabilities', col]
                
                current_ratio = safe_divide(current_assets, current_liabilities)
                current_ratio_list.append(current_ratio)
            
            trend_metrics['current_ratio_tm'] = current_ratio_list
        
        # Debt to Equity ratio as list
        if ('Total Debt' in financial_data.index and 
            'Stockholders Equity' in financial_data.index):
            
            debt_to_equity_list = []
            
            for col in available_years:
                total_debt = financial_data.loc['Total Debt', col]
                equity = financial_data.loc['Stockholders Equity', col]
                
                debt_equity_ratio = safe_divide(total_debt, equity)
                debt_to_equity_list.append(debt_equity_ratio)
            
            trend_metrics['debt_to_equity_tm'] = debt_to_equity_list
            
        # Revenue growth as list
        if 'Total Revenue' in financial_data.index:
            revenue_growth_list = []
            
            for i in range(len(available_years)):
                if i == 0:
                    # Most recent year - no growth calculation
                    revenue_growth_list.append(None)
                else:
                    current_rev = financial_data.loc['Total Revenue', available_years[i-1]]
                    prior_rev = financial_data.loc['Total Revenue', available_years[i]]
                    growth = safe_divide(current_rev - prior_rev, prior_rev)
                    revenue_growth_list.append(growth)
            
            trend_metrics['revenue_growth_tm'] = revenue_growth_list

        # Trend calculations as lists
        # SG&A to Revenue Ratio
        if ('Selling General And Administration' in financial_data.index and 
            'Total Revenue' in financial_data.index):
            
            sga_revenue_ratio_list = []
            
            for col in available_years:
                sga = financial_data.loc['Selling General And Administration', col]
                revenue = financial_data.loc['Total Revenue', col]
                
                ratio = safe_divide(sga, revenue)
                sga_revenue_ratio_list.append(ratio)
            
            trend_metrics['sga_revenue_ratio_tm'] = sga_revenue_ratio_list
        
        # Gross Profit Margin as list
        if 'Gross Profit' in financial_data.index and 'Total Revenue' in financial_data.index:
            gross_margin_list = []
            
            for col in available_years:
                gross_profit = financial_data.loc['Gross Profit', col]
                revenue = financial_data.loc['Total Revenue', col]
                
                margin = safe_divide(gross_profit, revenue)
                gross_margin_list.append(margin)
            
            trend_metrics['gross_profit_margin_tm'] = gross_margin_list
                    
        # Operating Profit Margin as list
        if 'Operating Income' in financial_data.index and 'Total Revenue' in financial_data.index:
            operating_margin_list = []
            
            for col in available_years:
                operating_income = financial_data.loc['Operating Income', col]
                revenue = financial_data.loc['Total Revenue', col]
                
                margin = safe_divide(operating_income, revenue)
                operating_margin_list.append(margin)
            
            trend_metrics['operating_profit_margin_tm'] = operating_margin_list

        # Net Profit Margin as list
        if 'Net Income' in financial_data.index and 'Total Revenue' in financial_data.index:
            net_margin_list = []
            
            for col in available_years:
                net_income = financial_data.loc['Net Income', col]
                revenue = financial_data.loc['Total Revenue', col]
                
                margin = safe_divide(net_income, revenue)
                net_margin_list.append(margin)
            
            trend_metrics['net_profit_margin_tm'] = net_margin_list
            
        # ROIC as list
        if ('Net Income' in financial_data.index and 
            'Stockholders Equity' in financial_data.index and 
            'Total Debt' in financial_data.index):
            
            roic_list = []
            
            for col in available_years:
                net_income = financial_data.loc['Net Income', col]
                equity = financial_data.loc['Stockholders Equity', col]
                debt = financial_data.loc['Total Debt', col]
                
                roic = safe_divide(net_income, equity + debt)
                roic_list.append(roic)
            
            trend_metrics['roic_tm'] = roic_list
                    
    except Exception as e:
        print(f"Error calculating trend metrics: {e}")
    
    return trend_metrics

# =============================================================================
# VALUATION CALCULATIONS
# =============================================================================

def calculate_all_valuations(company_stats):
    """Calculate all valuation metrics in one function"""
    valuations = {}
    market_rates = get_market_rates(company_stats)

    # P/E Ratio Calculation 
    try:
        current_price = company_stats.get('currentPrice')
        net_income = company_stats.get('netIncomeToCommon')
        shares_outstanding = company_stats.get('sharesOutstanding')   
        pe_ratio = company_stats.get('trailingPE')

        # Calculate P/E ratio if not available
        if not pe_ratio and net_income and shares_outstanding and current_price and net_income > 0:
            eps = net_income / shares_outstanding
            if eps > 0:
                valuations['pe_ratio'] = current_price / eps
        elif pe_ratio:
            valuations['pe_ratio'] = pe_ratio
            
    except Exception as e:
        print(f"Error in P/E ratio calculation: {e}")
        valuations['pe_ratio'] = None
    
    # PEGY Calculation
    try:
        pe_ratio = company_stats.get('trailingPE') or valuations.get('pe_ratio')
        dividend_yield = company_stats.get('dividendYield') or 0
        earnings_growth = company_stats.get('earningsGrowth') or 0
        
        # Ensure we have valid values for calculation
        if pe_ratio and pe_ratio > 0 and (earnings_growth is not None or dividend_yield is not None):
            total_growth = (earnings_growth or 0) + (dividend_yield or 0)
            if total_growth > 0:
                valuations['pegy'] = pe_ratio / total_growth
            else:
                valuations['pegy'] = None
        else:
            valuations['pegy'] = None
            
    except Exception as e:
        print(f"Error in PEGY calculation: {e}")
        valuations['pegy'] = None
                
    # DCF Valuation
    try:
        free_cash_flow = company_stats.get('freeCashflow')
        shares_outstanding = company_stats.get('sharesOutstanding')
        
        if free_cash_flow and free_cash_flow > 0 and shares_outstanding:
            revenue_growth = company_stats.get('revenueGrowth', market_rates['growth_rate'])
            earnings_growth = company_stats.get('earningsGrowth', market_rates['growth_rate'])
            growth_rate = min(max(revenue_growth, earnings_growth, market_rates['growth_rate']), market_rates['max_fcf_growth'])
            
            beta = company_stats.get('beta', market_rates['default_beta']) or market_rates['default_beta']
            discount_rate = market_rates['risk_free_rate'] + (beta * market_rates['market_risk_premium'])
            terminal_growth = market_rates['terminal_growth']
            
            # DCF Calculation
            years = market_rates['dcf_years']
            future_fcf = []
            current_fcf = free_cash_flow
            
            for year in range(1, years + 1):
                future_fcf.append(current_fcf * (1 + growth_rate) ** year)
            
            # Validate discount rate > terminal growth
            if discount_rate <= terminal_growth:
                discount_rate = terminal_growth + market_rates['min_discount_premium']
            
            terminal_value = future_fcf[-1] * (1 + terminal_growth) / (discount_rate - terminal_growth)
            
            present_value = 0
            for i, fcf in enumerate(future_fcf):
                present_value += fcf / ((1 + discount_rate) ** (i + 1))
            
            present_value += terminal_value / ((1 + discount_rate) ** years)
            valuations['dcf_value'] = present_value / shares_outstanding
        else:
            valuations['dcf_value'] = None
            
    except Exception as e:
        print(f"Error in DCF calculation: {e}")
        valuations['dcf_value'] = None
    
    # Graham Valuation
    try:
        eps = company_stats.get('trailingEps') or company_stats.get('epsTrailingTwelveMonths')
        book_value = company_stats.get('bookValue')
        
        if eps and book_value and eps > 0 and book_value > 0:
            # Graham formula: √(22.5 × EPS × Book Value per Share)
            valuations['graham_eps_bv'] = (market_rates['graham_constant'] * eps * book_value) ** 0.5
            
            earnings_growth = company_stats.get('earningsGrowth', market_rates['growth_rate']) or market_rates['growth_rate']
            if earnings_growth:
                # Graham growth formula: EPS × (8.5 + 2g)
                growth_rate = earnings_growth * 100
                valuations['graham_eps_eg'] = eps * (market_rates['graham_base_pe'] + 2 * growth_rate)
                
    except Exception as e:
        print(f"Error in Graham valuation: {e}")
        valuations['graham_eps_bv'] = None
        valuations['graham_eps_eg'] = None
    
    # Important Metrics
    try:
        price = company_stats.get('currentPrice')
        book_value = company_stats.get('bookValue')
        revenue = company_stats.get('totalRevenue')
        market_cap = company_stats.get('marketCap')
        fcf = company_stats.get('freeCashflow')
        
        if price and book_value and book_value > 0:
            valuations['price_to_book'] = safe_divide(price, book_value)
        else:
            valuations['price_to_book'] = None
            
        if market_cap and revenue and revenue > 0:
            valuations['price_to_sales'] = safe_divide(market_cap, revenue)
        else:
            valuations['price_to_sales'] = None
            
        if fcf and market_cap and market_cap > 0:
            valuations['fcf_yield'] = safe_divide(fcf, market_cap)
        else:
            valuations['fcf_yield'] = None
        
    except Exception as e:
        print(f"Error calculating advanced metrics: {e}")
    
    return valuations

# =============================================================================
# REPORT GENERATION
# =============================================================================

def generate_investment_insights(company_stats, valuations):
    """Generate investment insights based on all metrics"""
    insights = []
    current_price = company_stats.get('currentPrice')
    
    # Valuation Insights
    dcf_value = valuations.get('dcf_value')
    if dcf_value and current_price:
        if current_price < dcf_value * 0.7:
            insights.append("✅ STRONG UNDERVALUATION: Trading >30% below DCF intrinsic value")
        elif current_price < dcf_value * 0.9:
            insights.append("✅ MODERATE UNDERVALUATION: Trading 10-30% below DCF intrinsic value")
        elif current_price > dcf_value * 1.3:
            insights.append("🚫 SIGNIFICANT OVERVALUATION: Trading >30% above DCF intrinsic value")
    
    # Profitability Insights
    roe = company_stats.get('returnOnEquity')
    if roe is not None:
        if roe > BENCHMARKS['roe_strong']:
            insights.append("✅ EXCELLENT PROFITABILITY: ROE >15% indicates efficient capital use")
        elif roe < BENCHMARKS['roe_weak']:
            insights.append("🚫 WEAK PROFITABILITY: ROE <8% suggests poor capital efficiency")
    
    # Financial Health Insights
    debt_to_equity = company_stats.get('debtToEquity')
    if debt_to_equity is not None:
        if debt_to_equity > BENCHMARKS['debt_to_equity_high']:
            insights.append("🚫 HIGH LEVERAGE: Debt-to-Equity >1.0 indicates significant debt burden")
        elif debt_to_equity < BENCHMARKS['debt_to_equity_low']:
            insights.append("✅ CONSERVATIVE FINANCING: Low debt levels provide financial flexibility")
    
    # Cash Flow Insights
    fcf_yield = valuations.get('fcf_yield')
    if fcf_yield is not None:
        if fcf_yield > BENCHMARKS['fcf_yield_strong']:
            insights.append("✅ STRONG CASH GENERATION: FCF yield >8% indicates excellent cash flow")
        elif fcf_yield < BENCHMARKS['fcf_yield_weak']:
            insights.append("🚫 WEAK CASH GENERATION: FCF yield <3% suggests poor cash flow")
    
    # Growth Insights
    revenue_growth = company_stats.get('revenueGrowth')
    if revenue_growth is not None:
        if revenue_growth > BENCHMARKS['revenue_growth_strong']:
            insights.append("✅ STRONG GROWTH: Revenue growth >15% indicates expanding business")
        elif revenue_growth < BENCHMARKS['revenue_growth_slow']:
            insights.append("🚫 SLOW GROWTH: Revenue growth <5% suggests mature/declining business")
    
    # Overall Assessment
    positive_count = len([i for i in insights if '✅' in i])
    negative_count = len([i for i in insights if '🚫' in i])
    
    insights.append(f"\nSUMMARY: {positive_count} positive factors, {negative_count} concerns")
    
    if positive_count >= 4 and negative_count <= 1:
        insights.append("✅ STRONG INVESTMENT CANDIDATE: Multiple positive fundamentals")
    elif positive_count >= 2 and negative_count <= 2:
        insights.append("⚖️ MODERATE CANDIDATE: Mixed signals - requires deeper analysis")
    else:
        insights.append("🚫 HIGH-RISK CANDIDATE: Multiple concerning factors")
    
    return insights

def generate_report(ticker, company_stats, trend_metrics, valuations, insights):
    """Generate the main financial analysis report with insights"""
    
    # Calculate ev_mc_ratio
    market_cap = company_stats.get('marketCap')
    enterprise_value = company_stats.get('enterpriseValue')
    ev_mc_ratio = safe_divide(enterprise_value, market_cap) if (market_cap is not None and enterprise_value is not None) else None

    # Calculate net_debt
    total_debt = company_stats.get('totalDebt')
    total_cash = company_stats.get('totalCash')
    net_debt = total_debt - total_cash if (total_debt is not None and total_cash is not None) else None
    
    # Generate the report
    report_lines = []
    report_lines.append("=" * 80)
    report_lines.append(f"FINANCIAL ANALYSIS REPORT: {ticker}")
    report_lines.append("=" * 80)
    report_lines.append(f"Company: {company_stats.get('shortName', 'N/A')}")
    report_lines.append(f"Industry: {company_stats.get('industry', 'N/A')}")
    report_lines.append(f"Sector: {company_stats.get('sector', 'N/A')}")
    report_lines.append("")
    
    # EXECUTIVE SUMMARY SECTION
    report_lines.append("EXECUTIVE SUMMARY:")
    report_lines.append("-" * 40)
    report_lines.append(f"Market Cap: {format_value(company_stats.get('marketCap'), 'currency')}")
    report_lines.append(f"Enterprise Value: {format_value(company_stats.get('enterpriseValue'), 'currency')}")
    report_lines.append(f"EV/MC Ratio (cv): {format_value(ev_mc_ratio, 'ratio')}x")
    report_lines.append(f"Net Debt: {format_value(net_debt, 'currency')}")
    report_lines.append("")
    
    
    # PROFITABILITY & MARGINS SECTION
    report_lines.append("PROFITABILITY & MARGINS:")
    report_lines.append("-" * 40)

    report_lines.append(f"Gross Profit Margin Trend (tm): {format_trend_list(trend_metrics.get('gross_profit_margin_tm'), 'percentage')}")
    report_lines.append(f"Operating Profit Margin Trend (tm): {format_trend_list(trend_metrics.get('operating_profit_margin_tm'), 'percentage')}")
    report_lines.append(f"Net Profit Margin Trend (tm): {format_trend_list(trend_metrics.get('net_profit_margin_tm'), 'percentage')}")
    report_lines.append(f"EPS Growth: {format_value(company_stats.get('earningsGrowth'), 'growth')}")
    report_lines.append(f"Revenue Growth Trend (tm): {format_trend_list(trend_metrics.get('revenue_growth_tm'), 'growth')}")
    report_lines.append(f"SG&A to Revenue Ratio Trend (tm): {format_trend_list(trend_metrics.get('sga_revenue_ratio_tm'), 'percentage')}")
    report_lines.append("")
    
    # EFFICIENCY METRICS SECTION
    report_lines.append("OPERATIONAL EFFICIENCY:")
    report_lines.append("-" * 40)
    report_lines.append(f"ROE: {format_value(company_stats.get('returnOnEquity'), 'percentage', 1)}")
    report_lines.append(f"ROIC Trend (tm): {format_trend_list(trend_metrics.get('roic_tm'), 'percentage')}")
    report_lines.append("")
    
    # FINANCIAL HEALTH SECTION (Consolidated)
    report_lines.append("FINANCIAL HEALTH:")
    report_lines.append("-" * 40)
    report_lines.append(f"Current Ratio Trend (tm): {format_trend_list(trend_metrics.get('current_ratio_tm'), 'ratio')}")
    report_lines.append(f"Quick Ratio: {format_value(company_stats.get('quickRatio'), 'ratio', 2)}")
    report_lines.append(f"Debt-to-Equity Trend (tm): {format_trend_list(trend_metrics.get('debt_to_equity_tm'), 'ratio')}")
    report_lines.append(f"Free Cash Flow: {format_value(company_stats.get('freeCashflow'), 'currency')}")
    report_lines.append(f"FCF Yield (cv): {format_value(valuations.get('fcf_yield'), 'percentage')}")
    report_lines.append(f"Dividend Rate: {format_value(company_stats.get('dividendRate'), 'currency')}")
    report_lines.append(f"Dividend Yield: {format_value(safe_divide(company_stats.get('dividendYield'), 100), 'percentage', 2)}")
    report_lines.append("")

    # VALUATION SECTION
    report_lines.append("VALUATION METRICS:")
    report_lines.append("-" * 40)
    report_lines.append(f"Trailing P/E: {format_value(company_stats.get('trailingPE'), 'ratio', 1)}")
    report_lines.append(f"PEG Ratio: {format_value(company_stats.get('trailingPegRatio'), 'ratio', 2)}")
    report_lines.append(f"PE Ratio (cv): {format_value(valuations.get('pe_ratio'), 'ratio', 2)}")
    report_lines.append(f"PEGY (cv): {format_value(valuations.get('pegy'), 'ratio', 2)}")
    report_lines.append(f"Price/Book (cv): {format_value(valuations.get('price_to_book'), 'ratio', 2)}")
    report_lines.append(f"Price/Sales (cv): {format_value(valuations.get('price_to_sales'), 'ratio', 2)}")
    report_lines.append("")
    
    # VALUATION MODELS SECTION
    report_lines.append("INTRINSIC VALUATION MODELS:")
    report_lines.append("-" * 40)
    report_lines.append(f"Current Price: ${company_stats.get('currentPrice', 'N/A'):.2f}")
    report_lines.append(f"Discount Cashflow DCF (cv): {format_value(valuations.get('dcf_value'), 'currency')}")
    report_lines.append(f"Graham EPS EG (cv): {format_value(valuations.get('graham_eps_eg'), 'currency')}")
    report_lines.append(f"Graham EPS BV (cv): {format_value(valuations.get('graham_eps_bv'), 'currency')}")
    report_lines.append("")
    
    # COMPANY PROFILE SECTION
    report_lines.append("COMPANY PROFILE:")
    report_lines.append("-" * 40)
    report_lines.append(f"Website: {company_stats.get('website', 'N/A')}")
    report_lines.append(f"Beta: {format_value(company_stats.get('beta'), 'ratio', 2)}")
    
    report_lines.append("")
    report_lines.append("Business Summary:")
    report_lines.append(company_stats.get('longBusinessSummary', 'N/A'))
    report_lines.append("")
    report_lines.append("=" * 80)
    report_lines.append(f"Report generated on: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")
    report_lines.append("=" * 80)

     # INVESTMENT INSIGHTS SECTION
    report_lines.append("INVESTMENT INSIGHTS:")
    report_lines.append("-" * 40)
    for insight in insights:
        report_lines.append(insight)
    report_lines.append("")
    
    return "\n".join(report_lines)

# =============================================================================
# MAIN PROCESSING FUNCTION
# =============================================================================

def export_financial_data(ticker, financials_statement, output_text=None):
    """Export financial statements and analysis report"""
    try:
        if financials_statement is not None and not financials_statement.empty:
            os.makedirs("financial_data", exist_ok=True)

            '''
            # Stop csv file output
            # Export financial statements
            fs_filename = f"financial_data/{ticker}_financials.csv"
            financials_statement.to_csv(fs_filename, encoding='utf-8')
            print(f"Financial statements saved to: {fs_filename}")
            '''
            # Export analysis report
            if output_text:
                ar_filename = f"financial_data/{ticker}_analysis_report.txt"
                with open(ar_filename, "w", encoding="utf-8") as file:
                    file.write(output_text)
                print(f"Analysis report saved to: {ar_filename}")
            
            return True
        else:
            print(f"No financial data available for {ticker}")
            return False
    except Exception as e:
        print(f"Error exporting data for {ticker}: {str(e)}")
        return False

def process_ticker(ticker, save_all=False):
    """Process a single ticker and generate report"""
    try:
        company_stats, financials_statement = get_company_data(ticker)
        
        if not company_stats:
            print(f"No data found for ticker: {ticker}")
            return False
        
        trend_metrics = calculate_trend_metrics(financials_statement)
        valuations = calculate_all_valuations(company_stats)
        insights = generate_investment_insights(company_stats, valuations)
        report = generate_report(ticker, company_stats, trend_metrics, valuations, insights)
        
        # Display the report
        print(report)
        
        # Save data based on mode
        if save_all:
            # Auto-save mode for batch processing
            export_financial_data(ticker, financials_statement, report)
            print(f"Data auto-saved for {ticker}")
        else:
            # Interactive mode for single ticker
            save_data = input(f"\nSave financial data and report for {ticker}? (y/n): ").strip().lower()
            if save_data in ['y', 'yes']:
                export_financial_data(ticker, financials_statement, report)
                print(f"Data saved successfully for {ticker}")
            else:
                print(f"Data not saved for {ticker}")
        
        return True
        
    except Exception as e:
        print(f"Error processing {ticker}: {str(e)}")
        return False

def process_csv_file(csv_path):
    """Process a CSV file containing multiple tickers"""
    try:
        tickers_df = pd.read_csv(csv_path)
        tickers = tickers_df['ticker'].tolist() if 'ticker' in tickers_df.columns else tickers_df.iloc[:, 0].tolist()
        tickers = [str(t).strip().upper() for t in tickers if str(t).strip()]
        
        if not tickers:
            print("No valid tickers found in the CSV file.")
            return
        
        print(f"Found {len(tickers)} tickers in CSV file")
        
        # Ask if user wants to save all reports
        save_all = input("Save all reports automatically? (y/n): ").strip().lower() in ['y', 'yes']
        
        successful, failed = [], []
        
        for i, ticker in enumerate(tickers, 1):
            clear_output(wait=False)
            print(f"Processing {i}/{len(tickers)}: {ticker}")
            print("-" * 50)
            
            if process_ticker(ticker, save_all=save_all):
                successful.append(ticker)
            else:
                failed.append(ticker)
            
            # Small delay to avoid rate limiting
            if i < len(tickers):
                time.sleep(2)
        
        clear_output(wait=False)
        print("\n" + "="*50)
        print("BATCH PROCESSING COMPLETE!")
        print("="*50)
        print(f"Successful: {len(successful)} tickers")
        print(f"Failed: {len(failed)} tickers")
        if failed:
            print(f"Failed tickers: {', '.join(failed)}")
        print("="*50)
            
    except Exception as e:
        print(f"Error processing CSV file: {e}")

def main():
    """Main function to run the financial data analysis tool."""
    
    while True:
        user_input = input("\nEnter ticker symbol, CSV file path, or 'q' to quit: ").strip()
        
        if user_input.lower() in ['quit', 'exit', 'q']:
            print("Thank you for using the Financial Analysis Tool!")
            break
        
        if not user_input:
            continue
        
        # Process CSV file with multiple tickers
        if user_input.lower().endswith('.csv'):
            if os.path.exists(user_input):
                process_csv_file(user_input)
            else:
                print(f"CSV file not found: {user_input}")
        else:
            # Process single ticker (interactive mode)
            clear_output(wait=False)
            process_ticker(user_input.upper())

if __name__ == "__main__":
    main()


BATCH PROCESSING COMPLETE!
Successful: 10 tickers
Failed: 0 tickers



Enter ticker symbol, CSV file path, or 'q' to quit:  q


Thank you for using the Financial Analysis Tool!
